In [19]:
from keras.applications.nasnet import NASNetMobile as NASNet
from keras.applications.xception import Xception
from keras.applications.mobilenetv2 import MobileNetV2
from keras.utils import multi_gpu_model
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Input, Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import categorical_accuracy, top_k_categorical_accuracy, categorical_crossentropy
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.python.client import device_lib
import os
import numpy as np
import tensorflow as tf
import cv2
import time 
from keras import backend as K
from keras.backend.tensorflow_backend import set_session
import cProfile



In [23]:
K.set_learning_phase(0)

TRAIN_PATH = '../../data/img_data/small/recognized'
# TEST_PATH = '../../data/img_data/recognized'

# batch size used for validation and training
BATCH_SIZE = 680

# Number of passes over the data
TRAIN_EPOCHS = 100

# Percent of the data used for validation
VAL_SPLIT = 0.2

# size of our images
IMG_SIZE = (256, 256)

# determines the number of classes and total # of samples
walk_dir = os.walk(TRAIN_PATH)

# NUM_CLASSES = len(list(walk_dir)[0][1])
NUM_CLASSES = 340
NUM_SAMPLES = 34000
# for root, dirs, files in walk_dir:
#     NUM_SAMPLES += len(files)

In [3]:
def top_3_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)

In [18]:
single_threaded_model = MobileNetV2(input_shape=(IMG_SIZE[0], IMG_SIZE[1], 1), weights=None, classes=NUM_CLASSES)
model = multi_gpu_model(single_threaded_model, gpus=4)
model.compile(optimizer=Adam(lr=0.002), loss='categorical_crossentropy',
              metrics=[categorical_crossentropy, categorical_accuracy, top_3_accuracy])
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
lambda_13 (Lambda)              (None, 256, 256, 1)  0           input_4[0][0]                    
__________________________________________________________________________________________________
lambda_14 (Lambda)              (None, 256, 256, 1)  0           input_4[0][0]                    
__________________________________________________________________________________________________
lambda_15 (Lambda)              (None, 256, 256, 1)  0           input_4[0][0]                    
__________________________________________________________________________________________________
lambda_16 

In [20]:
# make transformers
train_datagen = ImageDataGenerator(
#         rotation_range=30,
#         shear_range=0.2,
#         zoom_range=[0.9, 1],
#         horizontal_flip=True,
#         fill_mode='nearest',
        validation_split=VAL_SPLIT)

In [24]:
train_generator = train_datagen.flow_from_directory(
        TRAIN_PATH,
        target_size=IMG_SIZE,
        color_mode="grayscale",
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        interpolation='nearest',
        subset='training')

validation_generator = train_datagen.flow_from_directory(
        TRAIN_PATH,
        target_size=IMG_SIZE,
        color_mode="grayscale",
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        interpolation='nearest',
        subset='validation')

Found 36483 images belonging to 340 classes.
Found 8960 images belonging to 340 classes.


In [ ]:
start = time.time() 
for i in range(50):
    next(train_generator)
print(time.time()-start)

In [14]:
# define callbacks
callbacks = [
    ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.5, patience=5,
                      min_delta=0.005, mode='max', cooldown=3, verbose=1)
]

In [ ]:
history = model.fit_generator(
            train_generator,
            steps_per_epoch=((1-VAL_SPLIT)*NUM_SAMPLES)//BATCH_SIZE,
            epochs=TRAIN_EPOCHS,
            validation_data=validation_generator,
            validation_steps=(VAL_SPLIT*NUM_SAMPLES)//BATCH_SIZE)

Epoch 1/100
850/850 [==============================] - 480s 565ms/step - loss: 5.8663 - categorical_crossentropy: 5.8663 - categorical_accuracy: 0.0062 - top_3_accuracy: 0.0197 - val_loss: 5.8219 - val_categorical_crossentropy: 5.8219 - val_categorical_accuracy: 0.0074 - val_top_3_accuracy: 0.0208
Epoch 2/100
850/850 [==============================] - 454s 534ms/step - loss: 5.8200 - categorical_crossentropy: 5.8200 - categorical_accuracy: 0.0073 - top_3_accuracy: 0.0204 - val_loss: 5.8173 - val_categorical_crossentropy: 5.8173 - val_categorical_accuracy: 0.0068 - val_top_3_accuracy: 0.0181
Epoch 3/100
850/850 [==============================] - 456s 537ms/step - loss: 5.8161 - categorical_crossentropy: 5.8161 - categorical_accuracy: 0.0070 - top_3_accuracy: 0.0196 - val_loss: 5.8128 - val_categorical_crossentropy: 5.8128 - val_categorical_accuracy: 0.0081 - val_top_3_accuracy: 0.0223
Epoch 4/100
850/850 [==============================] - 453s 533ms/step - loss: 5.8126 - categorical_cro